In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
os.getenv('TAVILY_API_KEY')

'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'

In [1]:
import os
import shutil
import sqlite3

import pandas as pd
import requests

db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file = "travel2.sqlite"
# The backup lets us restart for each tutorial section
backup_file = "travel2.backup.sqlite"
overwrite = False
if overwrite or not os.path.exists(local_file):
    response = requests.get(db_url)
    response.raise_for_status()  # Ensure the request was successful
    with open(local_file, "wb") as f:
        f.write(response.content)
    # Backup - we will use this to "reset" our DB in each section
    shutil.copy(local_file, backup_file)
# Convert the flights to present time for our tutorial
conn = sqlite3.connect(local_file)
cursor = conn.cursor()

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
).name.tolist()
tdf = {}
for t in tables:
    tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

example_time = pd.to_datetime(
    tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
).max()
current_time = pd.to_datetime("now").tz_localize(example_time.tz)
time_diff = current_time - example_time

tdf["bookings"]["book_date"] = (
    pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
    + time_diff
)

datetime_columns = [
    "scheduled_departure",
    "scheduled_arrival",
    "actual_departure",
    "actual_arrival",
]
for column in datetime_columns:
    tdf["flights"][column] = (
        pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT)) + time_diff
    )

for table_name, df in tdf.items():
    df.to_sql(table_name, conn, if_exists="replace", index=False)
del df
del tdf
conn.commit()
conn.close()

db = local_file  # We'll be using this local file as our DB in this tutorial

In [5]:
import re

import numpy as np
import openai
from langchain_core.tools import tool

response = requests.get(
    "https://storage.googleapis.com/benchmarks-artifacts/travel-db/swiss_faq.md"
)
response.raise_for_status()
faq_text = response.text

docs = [{"page_content": txt} for txt in re.split(r"(?=\n##)", faq_text)]


class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    def from_docs(cls, docs, oai_client):
        embeddings = oai_client.embeddings.create(
            model="text-embedding-3-small", input=[doc["page_content"] for doc in docs]
        )
        vectors = [emb.embedding for emb in embeddings.data]
        return cls(docs, vectors, oai_client)

    def query(self, query: str, k: int = 5) -> list[dict]:
        embed = self._client.embeddings.create(
            model="text-embedding-3-small", input=[query]
        )
        # "@" is just a matrix multiplication in python
        scores = np.array(embed.data[0].embedding) @ self._arr.T
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]


retriever = VectorStoreRetriever.from_docs(docs, openai.Client())


@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted.
    Use this before making any flight changes performing other 'write' events."""
    docs = retriever.query(query, k=2)
    return "\n\n".join([doc["page_content"] for doc in docs])

AuthenticationError: Error code: 401 - {'error': {'message': 'You must be a member of an organization to use the API. Please contact us through our help center at help.openai.com.', 'type': 'invalid_request_error', 'param': None, 'code': 'no_organization'}}

In [8]:
import sqlite3

# Create and populate the 'workouts' table
conn = sqlite3.connect('fitness.db')
cursor = conn.cursor()

# Create workouts table
cursor.execute('''
CREATE TABLE IF NOT EXISTS workouts (
    workout_id INTEGER PRIMARY KEY,
    name TEXT,
    goal TEXT,
    intensity TEXT,
    description TEXT
)
''')

# Insert sample data
workouts = [
    ('Full Body Strength', 'muscle gain', 'high', 'A high-intensity workout focusing on full-body strength.'),
    ('Cardio Blast', 'weight loss', 'medium', 'A medium-intensity cardio workout for weight loss.'),
    ('Yoga for Flexibility', 'flexibility', 'low', 'A low-intensity yoga workout to improve flexibility.'),
]

cursor.executemany('''
INSERT INTO workouts (name, goal, intensity, description)
VALUES (?, ?, ?, ?)
''', workouts)

conn.commit()
conn.close()

In [9]:
import sqlite3
from typing import Optional, List, Dict

# Assuming you have already created and populated a SQLite database named 'fitness.db'
# with a table 'workouts' that has columns: 'workout_id', 'name', 'goal', 'intensity', and 'description'.

# Define the search_workouts tool
def search_workouts(goal: str, intensity: Optional[str] = None) -> List[Dict]:
    """
    Search for workouts based on goal and intensity.

    Args:
        goal (str): The fitness goal, e.g., 'weight loss', 'muscle gain'.
        intensity (Optional[str]): The intensity level, e.g., 'low', 'medium', 'high'.

    Returns:
        List[Dict]: A list of workout dictionaries matching the search criteria.
    """
    conn = sqlite3.connect('fitness.db')
    cursor = conn.cursor()

    query = "SELECT * FROM workouts WHERE goal = ?"
    params = [goal]

    if intensity:
        query += " AND intensity = ?"
        params.append(intensity)

    cursor.execute(query, params)
    rows = cursor.fetchall()

    # Fetch column names to create a list of dictionaries
    column_names = [description[0] for description in cursor.description]
    workouts = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return workouts

# Example usage
if __name__ == "__main__":
    goal = "muscle gain"
    intensity = "high"
    results = search_workouts(goal, intensity)
    for workout in results:
        print(workout)

{'workout_id': 1, 'name': 'Full Body Strength', 'goal': 'muscle gain', 'intensity': 'high', 'description': 'A high-intensity workout focusing on full-body strength.'}
{'workout_id': 4, 'name': 'Full Body Strength', 'goal': 'muscle gain', 'intensity': 'high', 'description': 'A high-intensity workout focusing on full-body strength.'}
